In [10]:
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import dataset_processing
import torch.optim as optim
from torch.autograd import Variable
import torch


DATA_PATH = '../data'
TRAIN_DATA = 'images/train'
TEST_DATA = 'images/test'
TRAIN_IMG_FILE = 'train_images.txt'
TEST_IMG_FILE = 'test_images.txt'
TRAIN_LABEL_FILE = 'attribute_matrix_train.txt'
TEST_LABEL_FILE = 'attribute_matrix_test.txt'

../data/train_label_attributes.txt


In [2]:
NLABELS = 5

batch_size = 4

transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()])

In [3]:
dset_train = dataset_processing.DatasetProcessing(
    DATA_PATH, TRAIN_DATA, TRAIN_IMG_FILE, TRAIN_LABEL_FILE, transformations)
dset_train.label

../data/attribute_matrix_train.txt


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [4]:
dset_test = dataset_processing.DatasetProcessing(
    DATA_PATH, TEST_DATA, TEST_IMG_FILE, TEST_LABEL_FILE, transformations)
dset_test.label

../data/attribute_matrix_test.txt


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [5]:
train_loader = DataLoader(dset_train,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

In [6]:
test_loader = DataLoader(dset_test,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=4
                         )

In [7]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

from imutils import paths
from pathlib import Path
import os
import time
import copy
import pickle
from tqdm import tqdm

import pandas as pd
import matplotlib.pylab as plt
import numpy as np

# Local modules
from cub_tools.train import train_model
from cub_tools.visualize import imshow, visualize_model
from cub_tools.utils import save_model_dict, save_model_full, unpickle
from cub_tools.transforms import makeDefaultTransforms

In [8]:
# Script runtime options
model_name = 'resnet152'
model_func = models.resnet152
root_dir = '../data'
data_dir = os.path.join(root_dir,'images')
working_dir = os.path.join('../models/classification', model_name)
batch_size = 16
num_workers = 4
num_epochs = 40

In [9]:
# Setup the model and optimiser
model_ft = models.resnet152(pretrained=True)

In [10]:
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 312)


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device::', device)

Device:: cuda:0


In [12]:
model_ft = model_ft.to(device)

In [13]:
optimizer = optim.SGD(model_ft.parameters(), lr=0.0001)
criterion = nn.MultiLabelMarginLoss()



In [14]:
epochs = 3
for epoch in range(epochs):
    ### training phase
    total_training_loss = 0.0
    # total = 0.0
  
    for iter, traindata in enumerate(train_loader, 0):
        
        train_inputs, train_labels = traindata
        
        train_inputs, train_labels = Variable(train_inputs.cuda()), Variable(train_labels.cuda())


        optimizer.zero_grad()

        train_outputs = model_ft(train_inputs)
        loss = criterion(train_outputs, train_labels)
        loss.backward()
        optimizer.step()

        # total += train_labels.size(0)
        total_training_loss += loss.data
        print('Training Phase: Epoch: [%2d][%2d/%2d]\tIteration Loss: %.3f' %
              (iter, epoch, epochs, loss.data / train_labels.size(0)))
    ### testing phase
    for iter, testdata in enumerate(test_loader, 0):
        test_inputs, test_labels = testdata

        test_inputs, test_labels = Variable(test_inputs.cuda()), Variable(test_labels.cuda())


        test_outputs = model_ft(test_inputs)
        test_loss = criterion(test_outputs, test_labels)
        print('Testing Phase: Epoch: [%2d][%2d/%2d]\tIteration Loss: %.3f' %
              (iter, epoch, epochs, test_loss.data / test_labels.size(0)))

Training Phase: Epoch: [ 0][ 0/ 3]	Iteration Loss: 54.366
Training Phase: Epoch: [ 1][ 0/ 3]	Iteration Loss: 0.262
Training Phase: Epoch: [ 2][ 0/ 3]	Iteration Loss: 0.050
Training Phase: Epoch: [ 3][ 0/ 3]	Iteration Loss: 0.010
Training Phase: Epoch: [ 4][ 0/ 3]	Iteration Loss: 0.023
Training Phase: Epoch: [ 5][ 0/ 3]	Iteration Loss: 0.009
Training Phase: Epoch: [ 6][ 0/ 3]	Iteration Loss: 0.008
Training Phase: Epoch: [ 7][ 0/ 3]	Iteration Loss: 0.039
Training Phase: Epoch: [ 8][ 0/ 3]	Iteration Loss: 0.011
Training Phase: Epoch: [ 9][ 0/ 3]	Iteration Loss: 0.043
Training Phase: Epoch: [10][ 0/ 3]	Iteration Loss: 0.023
Training Phase: Epoch: [11][ 0/ 3]	Iteration Loss: 0.021
Training Phase: Epoch: [12][ 0/ 3]	Iteration Loss: 0.010
Training Phase: Epoch: [13][ 0/ 3]	Iteration Loss: 0.000
Training Phase: Epoch: [14][ 0/ 3]	Iteration Loss: 0.000
Training Phase: Epoch: [15][ 0/ 3]	Iteration Loss: 0.000
Training Phase: Epoch: [16][ 0/ 3]	Iteration Loss: 0.000
Training Phase: Epoch: [17][ 0

In [15]:
torch.save(model_ft.state_dict(), 'attribute_predictor.pth')

In [4]:
from torchvision import datasets, models
import torch
import torch.nn as nn
model = models.resnet152()
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 312)
model.load_state_dict(torch.load("attribute_predictor.pth"))

model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
from PIL import Image

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension
def predict(image_path, model, device):
    model.to(device)
    image = preprocess_image(image_path).to(device)
    with torch.no_grad():
        output = model(image)
    return output


In [15]:
image_path = '../data/images/test/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg'
output_tensor = predict(image_path, model, device)
print(output_tensor)


tensor([[ 1.3967e+01,  2.5079e+00,  2.7103e-01, -3.7042e-01, -2.4504e-01,
          2.6622e-01,  1.1368e-01, -7.4968e-01, -1.8576e-01,  3.1333e-01,
          3.6782e-01, -7.5500e-01,  5.7647e-01,  2.4568e-01, -4.7821e-01,
         -6.4534e-01,  4.6338e-02,  1.7054e-02,  2.5725e-01, -1.2282e-01,
         -7.0053e-01, -4.6231e-01, -5.8515e-01, -2.4366e-01,  6.4786e-03,
         -5.1854e-03, -3.6200e-01, -2.0179e-01,  4.3525e-01,  2.1155e-01,
         -2.1746e-01, -1.2490e-01, -2.2307e-01, -4.9908e-01, -2.1471e-01,
         -4.3102e-01, -1.2345e-01, -5.2144e-01, -5.6096e-01,  6.3291e-01,
          5.6718e-02,  1.8053e-01,  7.3808e-01, -9.2837e-01,  1.6734e-01,
         -9.3400e-03, -6.7779e-02, -4.6876e-01, -6.6566e-01,  7.1411e-01,
          4.0429e-01,  6.3919e-02,  3.5733e-01, -8.8047e-01,  2.7807e-01,
          3.3770e-01,  8.7016e-01, -2.0505e-01, -6.9547e-01, -1.5758e-01,
         -1.4525e-01, -3.8272e-01,  1.1038e-01, -4.0005e-01, -3.0158e-01,
         -4.7974e-01,  4.9542e-01, -2.

In [17]:
predicted_attributes = (output_tensor > 0.5).int()
print(predicted_attributes)

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0

# Далі там про файли


In [15]:
import pandas as pd
images_inf = pd.read_csv('../data/images.txt', sep=" ", header=None)
images_inf.columns = ["images_id", "file_name"]

train_test_info = pd.read_csv("../data/train_test_split.txt", sep =" ", header=None)
train_test_info.columns= ["images_id", "is_training"]

result = pd.merge(images_inf, train_test_info, on='images_id', how='inner')
print(result.head(5))

   images_id                                          file_name  is_training
0          1  001.Black_footed_Albatross/Black_Footed_Albatr...            0
1          2  001.Black_footed_Albatross/Black_Footed_Albatr...            1
2          3  001.Black_footed_Albatross/Black_Footed_Albatr...            0
3          4  001.Black_footed_Albatross/Black_Footed_Albatr...            1
4          5  001.Black_footed_Albatross/Black_Footed_Albatr...            1


# Creating file with train images names

In [21]:
train_images = result[result["is_training"]==1]["file_name"]
with open('train_images1.txt', 'w') as train_file:
    for image in train_images:
        train_file.write(f"{image}\n")

# Creating file with test images names


In [22]:
test_images = result[result["is_training"]==0]["file_name"]
with open('test_images1.txt', 'w') as test_file:
    for image in test_images:
        test_file.write(f"{image}\n")

# Create files with train label attributes

In [ ]:
import pandas as pd

data = pd.read_csv('../data/updated_data.txt', header = None, sep =" ")
data.columns = ["images_id", "attribute_id", "is_present"]
result = pd.merge(train_test_info, data, on='images_id', how='inner')

train_labels = result[result["is_training"] ==1]

with open("train_label_attributes.txt", "w") as train_file:
  for _, row in train_labels.iterrows():
    train_file.write(f"{row['images_id']} {row['attribute_id']} {row['is_present']}\n")




         images_id  is_training  attribute_id  is_present
312              2            1             1           0
313              2            1             2           0
314              2            1             3           0
315              2            1             4           0
316              2            1             5           0
...            ...          ...           ...         ...
3677539      11787            1           308           0
3677540      11787            1           309           0
3677541      11787            1           310           0
3677542      11787            1           311           1
3677543      11787            1           312           0

[1870128 rows x 4 columns]


In [39]:
test_labels = result[result["is_training"] ==0]

with open("test_label_attributes.txt", "w") as test_file:
  for _, row in test_labels.iterrows():
    test_file.write(f"{row['images_id']} {row['attribute_id']} {row['is_present']}\n")



In [42]:
data_train = pd.read_csv('train_label_attributes.txt', header = None, sep =" ")
data_train.columns = ["images_id", "attribute_id", "is_present"]

helper = data_train["images_id"].unique()
helper


array([    2,     4,     5, ..., 11782, 11784, 11787])

# Train rows

In [46]:
import numpy as np
import pandas as pd

# Read the file into a DataFrame
data = pd.read_csv('train_label_attributes.txt', sep=' ', header=None, names=['images_id', 'attribute_id', 'is_present'])

# Initialize an empty dictionary to store attributes
attributes_dict = {}

# Iterate over the data to populate the dictionary
for _, row in data.iterrows():
    image_id = row['images_id']
    if image_id not in attributes_dict:
        attributes_dict[image_id] = np.zeros(312, dtype =int)
    attributes_dict[image_id][row['attribute_id'] - 1] = row['is_present']

# Write to a text file
with open('attribute_matrix_train.txt', 'w') as f:
    for image_id, attributes in attributes_dict.items():
        f.write(' '.join(map(str, attributes)) + '\n')

print("Attribute matrix saved as text file.")

Attribute matrix saved as text file.


In [47]:
import numpy as np
import pandas as pd

# Read the file into a DataFrame
data = pd.read_csv('test_label_attributes.txt', sep=' ', header=None, names=['images_id', 'attribute_id', 'is_present'])

# Initialize an empty dictionary to store attributes
attributes_dict = {}

# Iterate over the data to populate the dictionary
for _, row in data.iterrows():
    image_id = row['images_id']
    if image_id not in attributes_dict:
        attributes_dict[image_id] = np.zeros(312, dtype =int)
    attributes_dict[image_id][row['attribute_id'] - 1] = row['is_present']

# Write to a text file
with open('attribute_matrix_test.txt', 'w') as f:
    for image_id, attributes in attributes_dict.items():
        f.write(' '.join(map(str, attributes)) + '\n')

print("Attribute matrix saved as text file.")

Attribute matrix saved as text file.
